In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import re
import importlib

module_path = Path(os.path.abspath("")).parent.parent
print(module_path)
sys.path.append(str(module_path))

from notebooks.inference import utils, rule_based_review_split

importlib.reload(utils)


processed_folder_path = f'{module_path}/data/processed'

/fsx/homes/Abdelrahman.Sadallah@mbzuai.ac.ae/mbzuai/review_rewrite
sdfsdfsdfsdfds fsdfsdf
**Therefore, the aspect score is: **0**
sdjgljsdgljslkdjgsd sldkjglsdkj sdgfdsgsd s b 2wq
/fsx/homes/Abdelrahman.Sadallah@mbzuai.ac.ae/mbzuai/review_rewrite
sdfsdfsdfsdfds fsdfsdf
**Therefore, the aspect score is: **0**
sdjgljsdgljslkdjgsd sldkjglsdkj sdgfdsgsd s b 2wq


In [2]:
all_reviews = []

# Step 1: Check if the processed folder exists
files = os.listdir(processed_folder_path)

# Step 2: Concat reviews from all sources
for file in files:
    file_path = os.path.join(processed_folder_path, file)
    if os.path.isfile(file_path) and file.endswith('.csv'):
        print(f'Processing file: {file}')
        df = pd.read_csv(file_path)
        print(f'Number of reviews from this source: {len(df)}')

        for index, row in df.iterrows():
            review = row['focused_review']

            if 'acl' in file:
                source = row['filename'].split('josn')[0]
            elif 'ARR' in file:
                source = row['paped_id']
            elif 'nips'in file or 'iclr' in file:
                source = row['review_id']
            
            all_reviews.append({'source': source, 'focused_review': review})

Processing file: iclr_reviews.csv


/tmp/ipykernel_3394774/94378738.py:11: DtypeWarning: Columns (1,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Number of reviews from this source: 43198
Processing file: reviewer2_ARR_2022_reviews.csv
Number of reviews from this source: 573
Processing file: acl2017_reviews.csv
Number of reviews from this source: 205
Processing file: nips_reviews.csv


/tmp/ipykernel_3394774/94378738.py:11: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,12,13,14,15,16,17,18,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Number of reviews from this source: 39684


In [3]:


    
all_reviews_df = pd.DataFrame(all_reviews)
all_reviews_df['focused_review'] = all_reviews_df['focused_review'].astype(str)

for index, row in all_reviews_df.iterrows():
    review = row['focused_review']
    review = utils.clean_text(review)
    all_reviews_df.at[index, 'focused_review'] = review


all_reviews_filtered = rule_based_review_split.split_and_filter(all_reviews_df,
                                                                review_key='focused_review',
                                                                consider_only_bullet_points=True, 
                                                                do_filter_typos=True,
                                                                filter_short_reviews=True,
                                                                exclude_long=False,
                                                                exclude_short=False)


all_reviews_filtered.to_csv(f'{module_path}/data/all_reviews.csv', index=False)

Initial number of reviews: 83660
Number of reviews after removing one-point zero-length review 83660


100%|██████████| 83660/83660 [00:54<00:00, 1541.21it/s]


Number of reviews with one point: 76933
Number of all points: 25020
Number of points after filtering typos and considering only bullet points: 17897
filtering reviwes, and only considering the ones with length of one STD away form mean.
Number of reviews before filtering: 5852
Number of the review points before filtering: 17897
mean: 65.08398055540034 std: 77.75540811373091 min: -12.671427558330564 max: 142.83938866913127


100%|██████████| 5852/5852 [00:00<00:00, 17577.21it/s]


Number of the reviews after filtering: 17897


In [4]:
import ast


all  = pd.read_csv(f'{module_path}/data/all_reviews.csv')

num_of_points = []
no_points = 0

sources_cnt = {}
sources_points_stats = {}
for i,r in all.iterrows():
    points = ast.literal_eval(r['split_review'])
    src = r['source'].split('_')[0]
    if len(points) == 1:
        no_points += 1
    else:
        sources_points_stats[src] = sources_points_stats[src] + len(points) if src in sources_points_stats else len(points)
    sources_cnt[src] = sources_cnt[src] + 1 if src in sources_cnt else 1
    num_of_points.append(len(points))


print(f'Number of reviews: {len(all)}')
print(f'Number of reviews with one point: {no_points}')
print(f'Number of reviews with more than one point: {len(all) - no_points}')
print(f'Precentage of reviews with one point: {no_points/len(all)}')
print(f'total number of points: {sum(num_of_points)}')
print(f'Average number of points: {sum(num_of_points)/len(num_of_points)}')
print(f'Max number of points: {max(num_of_points)}')
print(f'Min number of points: {min(num_of_points)}')

    


print(f'Number of samples of each source with more that one point')
for source, count in sources_cnt.items():
    print(f'{source}: {count} ')

print(f'Number of points for each source')
for source, count in sources_points_stats.items():
    print(f'{source}: {count}')


Number of reviews: 5852
Number of reviews with one point: 1301
Number of reviews with more than one point: 4551
Precentage of reviews with one point: 0.22231715652768283
total number of points: 17897
Average number of points: 3.058270676691729
Max number of points: 22
Min number of points: 1
Number of samples of each source with more that one point
ICLR: 1775 
ARR: 407 
ACL: 166 
NIPS: 3504 
Number of points for each source
ICLR: 3866
ARR: 1754
ACL: 534
NIPS: 10442


In [5]:
with open('/fsx/homes/Abdelrahman.Sadallah@mbzuai.ac.ae/mbzuai/review_rewrite/outputs/test_new_spliting.txt','w')as f:


    for i,r in all.sample(500).iterrows():
        focused_review = r['focused_review']
        splitted_reviews = ast.literal_eval(r['split_review'])
        f.write(f'Focused review:\n\n{r["focused_review"]}\n\n')
        for sr in splitted_reviews:
            f.write(f'Review Point: {sr}\n')
        f.write('='*50 + '\n\n')

In [6]:
len(all)

5852

In [7]:
utils.prepare_df_to_annotation(all,f'{module_path}/data/all_review_points.csv', total_points=0)

preparing the data frame for annotation


In [8]:
point = '''-- Typo p8 L719-721 left column: "the model is hard to" --> "it is hard for the model to"? & " hard to made" --> "hard to make"? 
'''
import re
quoted = r'["\'“”](.*?)["\'“”]'


from similarity.normalized_levenshtein import NormalizedLevenshtein
normalized_levenshtein = NormalizedLevenshtein()
for i,word in enumerate(point.split()):
            if word in ['->', '=>','-->','==>', '→', '⟶']:
                qouted_text = re.findall(quoted,point)
                print(qouted_text)
                for i, q in enumerate(qouted_text):
                    for j, q2 in enumerate(qouted_text):
                        if i != j:
                            print(f'Q1: {q} - Q2: {q2}')
                            print(normalized_levenshtein.distance(q, q2))
                # if i > 0 and normalized_levenshtein.distance( point[i+1], point[i-1]) < 0.8:
                #     print('Typo')

['the model is hard to', 'it is hard for the model to', ' hard to made', 'hard to make']
Q1: the model is hard to - Q2: it is hard for the model to
0.7037037037037037
Q1: the model is hard to - Q2:  hard to made
0.7
Q1: the model is hard to - Q2: hard to make
0.75
Q1: it is hard for the model to - Q2: the model is hard to
0.7037037037037037
Q1: it is hard for the model to - Q2:  hard to made
0.5925925925925926
Q1: it is hard for the model to - Q2: hard to make
0.6666666666666666
Q1:  hard to made - Q2: the model is hard to
0.7
Q1:  hard to made - Q2: it is hard for the model to
0.5925925925925926
Q1:  hard to made - Q2: hard to make
0.15384615384615385
Q1: hard to make - Q2: the model is hard to
0.75
Q1: hard to make - Q2: it is hard for the model to
0.6666666666666666
Q1: hard to make - Q2:  hard to made
0.15384615384615385
['the model is hard to', 'it is hard for the model to', ' hard to made', 'hard to make']
Q1: the model is hard to - Q2: it is hard for the model to
0.7037037037037

In [9]:
annotation  = pd.read_csv(f'{module_path}/data/annotated_review_points.csv')


annotation

,review_id,source,focused_review,review_point,human_actionability,human_specificity,human_verifiability,human_politeness,llm_actionability,llm_specificity,llm_verifiability,llm_politeness
0,905,ICLR_2023_2312,Weaknesses\n1. Literature Review\nThe paper re...,2. Unsupported claims and definitions The pape...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4732,NIPS_2020_1809,- Note sure whether the authors intend to rele...,- I'm missing a discussion why values for p di...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,612,ICLR_2021_971,Weaknesses\nThe proposed algorithm is not para...,2. The presented experimental results are not ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3649,NIPS_2020_791,There are several issues here which I would li...,* What is the relationship between the CDF and...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4288,NIPS_2020_251,* The NF assumption was not discussed as compa...,* NKF does not show markedly better performanc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
145,4008,NIPS_2020_404,- What strikes me as surprising is the direct ...,- The CR loss assumes all sampled points on th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,4072,NIPS_2020_1186,Theoretical Grounding: - Since the regret boun...,- No ablations of β (presumably a crucial desi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,1231,ICLR_2023_3550,Weaknesses\nI am not entirely sure about the u...,• A training algorithm is individually fair if...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,2297,ACL_2017_71_review.json,Weaknesses: -The explanation of methods in so...,- Based on section 2.4 it seems that topical r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
